**This notebook covered by the following [license](License.ipynb)  This note must not be removed**

# Documentation

[Link to official NetworkPolicy documentation](https://kubernetes.io/docs/concepts/services-networking/network-policies/)

## Activate Calico

`minikube start --network-plugin=cni --cni=calico`


## Alternativley Install Cilium in Minikube
https://docs.cilium.io/en/stable/gettingstarted/minikube/

**Warning** this might be highly version critical and needs testing if the minikube version is different from 1.3.1

Activate Berkeley Packet Filters

`minikube ssh -- sudo mount bpffs -t bpf /sys/fs/bpf`


**This might take some 6 minutes**

`kubectl create -f https://raw.githubusercontent.com/cilium/cilium/1.6.1/install/kubernetes/quick-install.yaml`


In [4]:
kubectl get pods -A 

NAMESPACE     NAME                                       READY   STATUS    RESTARTS        AGE
kube-system   calico-kube-controllers-58497c65d5-xm6d8   1/1     Running   0               5m14s
kube-system   calico-node-k6fpb                          1/1     Running   0               5m15s
kube-system   coredns-78fcd69978-mvc8k                   1/1     Running   0               5m14s
kube-system   etcd-minikube                              1/1     Running   0               5m26s
kube-system   kube-apiserver-minikube                    1/1     Running   0               5m26s
kube-system   kube-controller-manager-minikube           1/1     Running   0               5m26s
kube-system   kube-proxy-q9hbz                           1/1     Running   0               5m15s
kube-system   kube-scheduler-minikube                    1/1     Running   0               5m26s
kube-system   storage-provisioner                        1/1     Running   1 (4m43s ago)   5m24s


## Create a test

It will deploy a simple probe and echo server running with multiple replicas. The probe will only report readiness while it can successfully reach the echo server:

In [5]:
kubectl get pods

No resources found in default namespace.


The Apache example follows 
https://supergiant.io/blog/understanding-network-policies-in-kubernetes/
with the updated versions of the Cilium installation above

In [6]:
kubectl apply -f - <<EOF
apiVersion: apps/v1
kind: Deployment
metadata:
  name: sleep
spec:
  replicas: 1
  selector:
    matchLabels:
      app: sleep
  template:
    metadata:
      labels:
        app: sleep
    spec:
      containers:
      - name: sleep
        image: pstauffer/curl
        command: ["/bin/sleep", "3650d"]
        imagePullPolicy: IfNotPresent
EOF

deployment.apps/sleep created


In [7]:
kubectl get pods,svc

NAME                        READY   STATUS              RESTARTS   AGE
pod/sleep-56bc8d768-tj2hm   0/1     ContainerCreating   0          2s

NAME                 TYPE        CLUSTER-IP   EXTERNAL-IP   PORT(S)   AGE
service/kubernetes   ClusterIP   10.96.0.1    <none>        443/TCP   6m4s


In [8]:
SLEEP_POD=$(kubectl get pods | awk '/sleep-/{print $1}')
echo $SLEEP_POD

sleep-56bc8d768-tj2hm


In [9]:
kubectl exec $SLEEP_POD -- curl -m 5 -s heise.de

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


In [10]:
kubectl exec $SLEEP_POD -- curl -m 5 -s google.de

<HTML><HEAD><meta http-equiv="content-type" content="text/html;charset=utf-8">
<TITLE>301 Moved</TITLE></HEAD><BODY>
<H1>301 Moved</H1>
The document has moved
<A HREF="http://www.google.de/">here</A>.
</BODY></HTML>


In [11]:
dig heise.de


; <<>> DiG 9.16.1-Ubuntu <<>> heise.de
;; global options: +cmd
;; Got answer:
;; ->>HEADER<<- opcode: QUERY, status: NOERROR, id: 20389
;; flags: qr rd ra; QUERY: 1, ANSWER: 1, AUTHORITY: 0, ADDITIONAL: 1

;; OPT PSEUDOSECTION:
; EDNS: version: 0, flags:; udp: 512
;; QUESTION SECTION:
;heise.de.			IN	A

;; ANSWER SECTION:
heise.de.		34424	IN	A	193.99.144.80

;; Query time: 18 msec
;; SERVER: 192.168.49.1#53(192.168.49.1)
;; WHEN: Wed Dec 01 14:03:56 UTC 2021
;; MSG SIZE  rcvd: 53



## Networkpolicy


In [12]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: test-np
  namespace: default
spec:
  podSelector:
    matchLabels:
      app: sleep
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          project: dev
    - podSelector:
        matchLabels:
          role: frontend
    ports:
    - protocol: TCP
      port: 80
  egress:
  - to:
    - ipBlock:
        cidr: 193.99.144.80/32
    ports:
    - protocol: TCP
      port: 80
EOF

networkpolicy.networking.k8s.io/test-np created


In [13]:
kubectl exec $SLEEP_POD -- curl -m 5 -s heise.de
echo $?

command terminated with exit code 28
28


In [14]:
kubectl exec $SLEEP_POD -- curl -m 5  -s google.de
echo $?

command terminated with exit code 28
28


In [16]:
kubectl exec $SLEEP_POD -- curl -m 5 -s 193.99.144.80
echo $?

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>
0


In [18]:
kubectl exec $SLEEP_POD -- curl -m 5  -s 172.217.19.67

command terminated with exit code 28


: 28

# Include DNS

In [19]:
kubectl apply -f - <<EOF
apiVersion: networking.k8s.io/v1
kind: NetworkPolicy
metadata:
  name: test-np
  namespace: default
spec:
  podSelector:
    matchLabels:
      app: sleep
  policyTypes:
  - Ingress
  - Egress
  ingress:
  - from:
    - namespaceSelector:
        matchLabels:
          project: dev
    - podSelector:
        matchLabels:
          role: frontend
    ports:
    - protocol: TCP
      port: 80
  egress:
  - to:
    - ipBlock:
        cidr: 193.99.144.80/32
    ports:
    - protocol: TCP
      port: 80
  - ports:
    - port: 53
      protocol: UDP
    - port: 53
      protocol: TCP
EOF

networkpolicy.networking.k8s.io/test-np configured


In [20]:
kubectl exec $SLEEP_POD -- curl -m 5 -s heise.de

<!DOCTYPE HTML PUBLIC "-//IETF//DTD HTML 2.0//EN">
<html><head>
<title>301 Moved Permanently</title>
</head><body>
<h1>Moved Permanently</h1>
<p>The document has moved <a href="https://www.heise.de/">here</a>.</p>
</body></html>


In [31]:
if  kubectl exec $SLEEP_POD -- curl -v -m 5 -s google.de 
then
  echo connection
else
  echo no connection
fi

* Rebuilt URL to: google.de/
*   Trying 172.217.19.67...
* TCP_NODELAY set
*   Trying 2a00:1450:4005:80b::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4005:80b::2003: Address not available
*   Trying 2a00:1450:4005:80b::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4005:80b::2003: Address not available
*   Trying 2a00:1450:4005:80b::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4005:80b::2003: Address not available
*   Trying 2a00:1450:4005:80b::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4005:80b::2003: Address not available
*   Trying 2a00:1450:4005:80b::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4005:80b::2003: Address not available
*   Trying 2a00:1450:4005:80b::2003...
* TCP_NODELAY set
* Immediate connect fail for 2a00:1450:4005:80b::2003: Address not available
* Connection timed out after 5001 milliseconds
* stopped the pause stream!
* Closing connection 0
command terminated with

In [ ]:
kubectl get pods 

# CNI

In [22]:
ls /etc/cni/net.d

10-calico.conflist  100-crio-bridge.conf  200-loopback.conf  calico-kubeconfig


In [24]:
cat /etc/cni/net.d/10-calico.conflist

{
  "name": "k8s-pod-network",
  "cniVersion": "0.3.1",
  "plugins": [
    {
      "type": "calico",
      "log_level": "info",
      "log_file_path": "/var/log/calico/cni/cni.log",
      "datastore_type": "kubernetes",
      "nodename": "minikube",
      "mtu": 0,
      "ipam": {
          "type": "calico-ipam"
      },
      "policy": {
          "type": "k8s"
      },
      "kubernetes": {
          "kubeconfig": "/etc/cni/net.d/calico-kubeconfig"
      }
    },
    {
      "type": "portmap",
      "snat": true,
      "capabilities": {"portMappings": true}
    },
    {
      "type": "bandwidth",
      "capabilities": {"bandwidth": true}
    }
  ]
}

In [25]:
cat /etc/cni/net.d/calico-kubeconfig

# Kubeconfig file for Calico CNI plugin.
apiVersion: v1
kind: Config
clusters:
- name: local
  cluster:
    server: https://[10.96.0.1]:443
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ETXdNVEE1TlRRMU1Gb1hEVE14TURJeU9EQTVOVFExTUZvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBTkh5Cm1SRm1qaG0wdWpKRXd4ZXp2VWZvL0o2Q1Mvbkk3MS9rQlZxVEVTY3ZoZFpkYjdUazl3T1Bueng3VW9UQUIwZXoKZ3AxUjdnVFBnZnJSNGNyKy9Lc08vWExydk9CNEFoOXM0RE1lL1B5OTVyZE1iNmlDUytZMDV1VldkQ3R0bGhrSwpjM3lmblhPS29teXFaVUw0dzJLTFQza1I2clBHVXZ6M3RMYU9lYlpYc2RYNklqZXBaOHVROXZkYldXYVVVS25uCk5jUHdLc3VCNmNra1hibnBmaXRXSWovVnEwbFdZWU5OOXVJZjg0TWFqY2ZjS2twRUpoY2g4SHd2aWdtZGRiWHkKL2hnNXd0VGF3RCtUUWp5d09MVFpIMkg5bEVodGNQMS9PeE9nOU56WitaQW1CcDZFelFJbmFJMCt0YVM2WW1FbQppQ0RzUVlEYnV0bXpFWnliVEJjQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NH

In [26]:
ip a

1: lo: <LOOPBACK,UP,LOWER_UP> mtu 65536 qdisc noqueue state UNKNOWN group default qlen 1000
    link/loopback 00:00:00:00:00:00 brd 00:00:00:00:00:00
    inet 127.0.0.1/8 scope host lo
       valid_lft forever preferred_lft forever
2: tunl0@NONE: <NOARP,UP,LOWER_UP> mtu 1480 qdisc noqueue state UNKNOWN group default qlen 1000
    link/ipip 0.0.0.0 brd 0.0.0.0
    inet 10.244.120.64/32 scope global tunl0
       valid_lft forever preferred_lft forever
3: docker0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN group default 
    link/ether 02:42:0f:4d:e2:95 brd ff:ff:ff:ff:ff:ff
    inet 172.17.0.1/16 brd 172.17.255.255 scope global docker0
       valid_lft forever preferred_lft forever
4: cni0: <NO-CARRIER,BROADCAST,MULTICAST,UP> mtu 1500 qdisc noqueue state DOWN group default qlen 1000
    link/ether 7e:96:62:01:ad:e4 brd ff:ff:ff:ff:ff:ff
    inet 10.85.0.1/16 brd 10.85.255.255 scope global cni0
       valid_lft forever preferred_lft forever
39: eth0@if40: <BROAD

In [28]:
curl -m 5 -k  https://10.96.0.1:443

{
  "kind": "Status",
  "apiVersion": "v1",
  "metadata": {
    
  },
  "status": "Failure",
  "message": "forbidden: User \"system:anonymous\" cannot get path \"/\"",
  "reason": "Forbidden",
  "details": {
    
  },
  "code": 403
}